In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata

In [3]:
import numpy as np
import pandas as pd

In [4]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "behavioral_data.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "abdelrahmanmahmoud22/behavioral-data",
  file_path,
)

print("First 5 records:", df.head())

First 5 records:   trans_date_trans_time            cc_num                      merchant  \
0   2019-01-01 00:00:18  2703186189652095          Rippin, Kub and Mann   
1   2019-01-01 00:00:44      630423337322     Heller, Gutmann and Zieme   
2   2019-01-01 00:00:51    38859492057661                Lind-Buckridge   
3   2019-01-01 00:01:16  3534093764340240  Kutch, Hermiston and Farrell   
4   2019-01-01 00:03:06   375534208663984                 Keeling-Crist   

        category     amt  gender            city state  city_pop  is_fraud  \
0       misc_net    4.97       1  Moravian Falls    NC      3495         0   
1    grocery_pos  107.25       1          Orient    WA       149         0   
2  entertainment  220.10       0      Malad City    ID      4154         0   
3  gas_transport   45.00       0         Boulder    MT      1939         0   
4       misc_pos   41.97       0        Doe Hill    VA        99         0   

   age           Region              job_category  
0   31     

In [ ]:
df_fraud = df[df['is_fraud'] == 1].copy()
df_non_fraud = df[df['is_fraud'] == 0].copy()

print(f"Number of fraud transactions in df: {len(df_fraud)}")
print(f"Number of non-fraud transactions in df: {len(df_non_fraud)}")

Number of fraud transactions in df: 9591
Number of non-fraud transactions in df: 1823081


In [11]:
# Get the unique credit card numbers
unique_cc_nums = df_fraud['cc_num'].unique()

# Initialize a list to store generated dataframes
generated_dataframes = []

# Count how many cards will be processed (with >= 5 fraud samples)
eligible_cards = [cc_num for cc_num in unique_cc_nums if len(df_fraud[df_fraud['cc_num'] == cc_num]) >= 5]
total_operations = len(eligible_cards)

print(f"Total operations to run: {total_operations}\n")

# Iterate with index
for i, cc_num in enumerate(eligible_cards, start=1):
    print(f"[{i}/{total_operations}] Generating data for cc_num: {cc_num}")

    # Filter the fraud and non-fraud data for the current credit card number
    cc_df = df_fraud[df_fraud['cc_num'] == cc_num].copy()
    cc_df_non_fraud = df_non_fraud[df_non_fraud['cc_num'] == cc_num].copy()

    # Define metadata for the current credit card dataframe
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data=cc_df)
    metadata.set_primary_key(None)
    metadata.update_column('merchant', sdtype='categorical')
    metadata.update_column('city', sdtype='categorical')
    metadata.update_column('state', sdtype='categorical')
    metadata.update_column('city_pop', sdtype='numerical')

    # Initialize CTGAN synthesizer
    synthesizer = CTGANSynthesizer(
        metadata=metadata,
        embedding_dim=128,
        generator_lr=2e-4,
        discriminator_lr=2e-4,
        batch_size=100,
        epochs=100,
        cuda=True,
        verbose=True
    )

    try:
        synthesizer.fit(cc_df)

        num_samples_to_generate = int(len(cc_df_non_fraud) * 0.7)  # Tune as needed

        synthetic_data = synthesizer.sample(num_rows=num_samples_to_generate)

        generated_dataframes.append(synthetic_data)

        print(f"> Generated {len(synthetic_data)} records for cc_num: {cc_num}\n")

    except Exception as e:
        print(f"> Error on cc_num {cc_num}: {e}\n")
        continue


Total operations to run: 931

[1/931] Generating data for cc_num: 4613314721966


Gen. (0.52) | Discrim. (0.04):   6%|▌         | 6/100 [00:00<00:01, 51.29it/s]

Gen. (1.59) | Discrim. (0.53): 100%|██████████| 100/100 [00:02<00:00, 39.25it/s]


> Generated 1530 records for cc_num: 4613314721966

[2/931] Generating data for cc_num: 340187018810220


Gen. (0.62) | Discrim. (-0.09): 100%|██████████| 100/100 [00:02<00:00, 43.69it/s]


...

> Generated 1527 records for cc_num: 374125201044065

[930/931] Generating data for cc_num: 4716561796955522


Gen. (0.03) | Discrim. (-0.15): 100%|██████████| 100/100 [00:02<00:00, 34.50it/s]


> Generated 3047 records for cc_num: 4716561796955522

[931/931] Generating data for cc_num: 2242176657877538


Gen. (-0.34) | Discrim. (-0.10): 100%|██████████| 100/100 [00:02<00:00, 40.00it/s]

> Generated 0 records for cc_num: 2242176657877538



In [12]:
if generated_dataframes:
    final_synthetic_df = pd.concat(generated_dataframes, ignore_index=True)
    final_synthetic_df.to_csv("synthetic_fraud_data.csv", index=False)
    print(f"\n✅ Exported {len(final_synthetic_df)} synthetic rows to 'synthetic_fraud_data.csv'")
else:
    print("\n⚠️ No synthetic data was generated.")


✅ Exported 1171605 synthetic rows to 'synthetic_fraud_data.csv'


In [13]:
if generated_dataframes:
    final_synthetic_df = pd.concat(generated_dataframes, ignore_index=True)

    # Combine with original data
    all_data = pd.concat([df_fraud, df_non_fraud, final_synthetic_df], ignore_index=True)

    # Export
    all_data.to_csv("combined_creditcard_data.csv", index=False)

    print(f"\n✅ Exported {len(all_data)} total rows to 'combined_creditcard_data.csv'")
    print(f" - Original fraud rows: {len(df_fraud)}")
    print(f" - Original non-fraud rows: {len(df_non_fraud)}")
    print(f" - Synthetic fraud rows: {len(final_synthetic_df)}")
else:
    print("\n⚠️ No synthetic data was generated.")



✅ Exported 3004277 total rows to 'combined_creditcard_data.csv'
 - Original fraud rows: 9591
 - Original non-fraud rows: 1823081
 - Synthetic fraud rows: 1171605


In [14]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3004277 entries, 0 to 3004276
Data columns (total 13 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   trans_date_trans_time  object 
 1   cc_num                 int64  
 2   merchant               object 
 3   category               object 
 4   amt                    float64
 5   gender                 int64  
 6   city                   object 
 7   state                  object 
 8   city_pop               int64  
 9   is_fraud               int64  
 10  age                    int64  
 11  Region                 object 
 12  job_category           object 
dtypes: float64(1), int64(5), object(7)
memory usage: 298.0+ MB


In [15]:
# Check fraud label balance
print("Class distribution in combined data:")
print(all_data['is_fraud'].value_counts())

# Optional: as percentages
print("\nPercentage distribution:")
print(all_data['is_fraud'].value_counts(normalize=True) * 100)


Class distribution in combined data:
is_fraud
0    1823081
1    1181196
Name: count, dtype: int64

Percentage distribution:
is_fraud
0    60.682853
1    39.317147
Name: proportion, dtype: float64


In [16]:
# Save trained model
model_path = f"ctgan.pkl"
synthesizer.save(model_path)
print(f"> Saved model to {model_path}")

> Saved model to ctgan.pkl


In [17]:
# Group by 'cc_num' and count the transactions
transaction_counts = all_data['cc_num'].value_counts()

print("\nNumber of transactions per credit card number:")
transaction_counts


Number of transactions per credit card number:


cc_num
30270432095985         7455
6538441737335434       7453
4642255475285942       7447
36722699017270         7446
4745996322265          7445
                       ... 
4225628813173670          7
4352307151555405069       7
4714017207228610634       7
340187018810220           7
4295296907373             6
Name: count, Length: 992, dtype: int64

In [18]:
from sdv.evaluation.single_table import evaluate_quality

# Use only the fraud data as base (real vs synthetic fraud)
synthetic_data = final_synthetic_df.copy()

# Evaluate
quality_report = evaluate_quality(
    real_data=df_fraud,
    synthetic_data=synthetic_data,
    metadata=metadata  # Use the last valid metadata or recreate it with all fraud columns
)

# Print score summary
print("\n=== Synthetic Data Quality Report ===")
print(f"Overall Quality Score: {quality_report.get_score():.3f}")
print(quality_report.get_details())


Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 13/13 [00:02<00:00,  6.08it/s]|
Column Shapes Score: 92.02%

(2/2) Evaluating Column Pair Trends: |██████████| 78/78 [00:05<00:00, 13.41it/s]|
Column Pair Trends Score: 80.24%

Overall Score (Average): 86.13%


=== Synthetic Data Quality Report ===
Overall Quality Score: 0.861


TypeError: BaseReport.get_details() missing 1 required positional argument: 'property_name'